# Imports

In [1]:
from datasets import load_dataset, load_from_disk
import seaborn as sns
import os
from huggingface_hub import notebook_login

print("Modlues imported sucessfully")

Modlues imported sucessfully


In [2]:
notebook_login()

print("Login successful")

Login successful


# Analysis of the dataset
## Dataset features
### Core Study Information
- **nct_id:** Unique study identifier (NCT########)  
- **brief_title:** Concise study title  
- **official_title:** Complete formal study title  
- **study_type:** Study design type (Interventional, Observational, etc.)  
- **phases:** Clinical trial phases (Phase I, II, III, IV)  
- **overall_status:** Current study status  

### Study Design & Population
- **enrollment_count:** Target or actual enrollment number  
- **minimum_age / maximum_age:** Age eligibility bounds  
- **sex:** Gender eligibility (All, Male, Female)  
- **healthy_volunteers:** Whether healthy volunteers are accepted  
- **eligibility_criteria:** Detailed inclusion/exclusion criteria  
- **study_population:** Target population description  

### Clinical Context
- **conditions:** Medical conditions studied  
- **keywords:** Study-related keywords  
- **brief_summary:** Study purpose and rationale  
- **detailed_description:** Comprehensive study description  
- **primary_outcomes / secondary_outcomes:** Measured endpoints  

### Organizational Information
- **lead_sponsor:** Primary study sponsor  
- **collaborators:** Additional supporting organizations  
- **locations:** Study sites with geographic coordinates  
- **overall_officials:** Principal investigators and study officials  

### Temporal Data
- **study_first_submit_date:** Initial submission to ClinicalTrials.gov  
- **start_date:** Study start date  
- **primary_completion_date:** Primary endpoint completion date  
- **completion_date:** Overall study completion date  
- **last_update_submit_date:** Most recent data update date  

### Enhanced Features
- **brief_summary_embedding:** 768-dimensional semantic embedding of study summary  
- **eligibility_criteria_embedding:** 768-dimensional embedding of eligibility text  

## Load dataset
if dataset is present on your device, then load it from disk, otherwise, load it from internet and save it on disk

In [3]:
path_to_ds = "./clinical-dataset" # your path to the dataset

print(f"Path is set to: \"{path_to_ds}\"")

Path is set to: "./clinical-dataset"


In [4]:

if os.path.exists(path_to_ds):
    dataset = load_from_disk(path_to_ds)
else:
    dataset = load_dataset("louisbrulenaudet/clinical-trials")
    dataset.save_to_disk(path_to_ds)

train_data = dataset["train"]

print(f"Dataset size: {len(train_data)}")
print(f"Features: {train_data.features}")


data/train-00003-of-00008.parquet:   0%|          | 0.00/222M [00:00<?, ?B/s]

data/train-00004-of-00008.parquet:   0%|          | 0.00/220M [00:00<?, ?B/s]

data/train-00005-of-00008.parquet:   0%|          | 0.00/223M [00:00<?, ?B/s]

data/train-00006-of-00008.parquet:   0%|          | 0.00/222M [00:00<?, ?B/s]

data/train-00007-of-00008.parquet:   0%|          | 0.00/222M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/541897 [00:00<?, ? examples/s]

Saving the dataset (0/8 shards):   0%|          | 0/541897 [00:00<?, ? examples/s]

Dataset size: 541897
Features: {'nct_id': Value('string'), 'updated_at': Value('timestamp[us]'), 'brief_title': Value('string'), 'official_title': Value('string'), 'acronym': Value('string'), 'study_type': Value('string'), 'overall_status': Value('string'), 'study_first_submit_date': Value('timestamp[ms]'), 'start_date': Value('timestamp[ms]'), 'primary_completion_date': Value('timestamp[ms]'), 'completion_date': Value('timestamp[ms]'), 'phases': List(Value('string')), 'enrollment_count': Value('float64'), 'minimum_age': Value('float64'), 'maximum_age': Value('float64'), 'sex': Value('string'), 'healthy_volunteers': Value('bool'), 'brief_summary': Value('string'), 'detailed_description': Value('string'), 'eligibility_criteria': Value('string'), 'lead_sponsor_name': Value('string'), 'lead_sponsor_class': Value('string'), 'org_study_id_info': {'id': Value('string'), 'link': Value('string'), 'type': Value('string')}, 'why_stopped': Value('string'), 'expanded_access_info': {'hasExpandedAcc